In [4]:
import pandas as pd
import altair as alt

import spacy
import re
import string

import gensim 
from gensim.models import LdaModel

import gensim.corpora as corpora
from gensim.corpora import Dictionary

from gensim import models

import pyLDAvis.gensim

import timeit
from pandarallel import pandarallel

pd.set_option('display.max_colwidth', 100)

In [5]:
# import briefings 
briefings_df = pd.read_csv('../data/whtfb/all_briefings.csv')

In [6]:
# Load sEnglish spacy model and stop words
nlp = spacy.load("en_core_web_sm")
from spacy.lang.en.stop_words import STOP_WORDS

In [7]:
# function for preprocessing each paragraph of transcript speach 
def preprocess(text, 
               min_token_len = 2, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']): 
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text 
    and return a preprocessed list of strings. 
    
    Parameters
    -------------
    text : (str) 
        the text to be preprocessed
    min_token_len : (int) 
        min_token_length required
    irrelevant_pos : (list) 
        a list of irrelevant pos tags
    
    Returns
    -------------
    (list) the preprocessed text as a list of strings
    """
    
    # convert input string to lowercase
    text = text.lower()
    
    # remove some of the obviously irrelevant character patterns
    text = re.sub('from', '', text)
    text = re.sub('subject', '', text)
    text = re.sub('organization', '', text)
    text = re.sub('distribution', '', text)
    text = re.sub('lines', '', text)
    text = re.sub('article', '', text)

    # remove multiple whitespace characters
    text = re.sub(r'\s+',' ', text)
    
    # tokenize with spacy, exluding stop words, short tokens, 
    # irrelevant POS, emails, urls, and strings containing 
    # non-alphanumeric chars
    
    doc = nlp(text)
    token_list = []
    for token in doc:
        if token.is_stop == False and len(token.text)>=min_token_len \
            and token.pos_ not in irrelevant_pos and token.like_email == False \
            and token.like_url == False and token.text.isalnum():
            token_list.append(token.lemma_)
        
    return token_list

#### Pre-process the raw text cells:

In [8]:
# parallelize preprocessing to reduce execution time
pandarallel.initialize(verbose=False)

# apply preprocessor to each row of text
briefings_df['pp_text'] = briefings_df.text.parallel_apply(preprocess)

In [9]:
briefings_df['pp_text']

0       [thank, thank, begin, like, extend, deep, condolence, victim, family, milwaukee, wisconsin, toda...
1       [lot, people, think, turn, good, thing, number, priority, standpoint, health, safety, american, ...
2       [total, 15, take, japan, hear, american, citizen, quarantine, get, well, feel, obligation, 42, f...
3       [china, know, start, speak, president, xi, great, talk, work, work, hard, count, report, come, c...
4       [bring, specialist, regarded, specialist, tomorrow, work, state, department, talented, want, und...
                                                       ...                                                 
7171    [follow, approach, follow, approach, think, million, people, dead, sweden, have, lot, difficulty...
7172    [minimum, number, reason, number, think, american, people, discipline, think, possible, guess, w...
7173    [okay, number, cut, half, people, right, unacceptable, people, spend, money, stimulus, care, com...
7174    [say, get, work, get

#### Create dictionary and document-term co-occurence matrix

In [23]:
# build dictionary
corpus = briefings_df['pp_text'].tolist()
dictionary = corpora.Dictionary(corpus)
len(dictionary)

6794

In [24]:
# build dictionary and filter extremes,
# removing tokens that appear in fewer than 2 paragraphs AND more than 5% of paragraphs
dictionary = corpora.Dictionary(corpus)
dictionary.filter_extremes(no_below = 2, no_above = 0.05)
len(dictionary)

4315

In [25]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

#### Build the topic model

In [26]:
num_topics = [7]
passes = [10]

for num in num_topics:
    for num_pass in passes:
        lda = models.LdaModel(corpus=doc_term_matrix, 
                      id2word=dictionary, 
                      num_topics=num, 
                      passes=num_pass,
                      random_state=123)
        print("Topics =",num,", Passes =", num_pass)
        print(lda.print_topics(num_words=8))
        print()
        print()

Topics = 7 , Passes = 10
[(0, '0.018*"healthcare" + 0.015*"supply" + 0.014*"mask" + 0.012*"worker" + 0.010*"government" + 0.010*"force" + 0.009*"national" + 0.009*"task"'), (1, '0.032*"ventilator" + 0.015*"million" + 0.015*"thousand" + 0.011*"testing" + 0.009*"inaudible" + 0.009*"sir" + 0.009*"mike" + 0.008*"send"'), (2, '0.031*"business" + 0.021*"money" + 0.020*"company" + 0.016*"billion" + 0.016*"small" + 0.014*"pay" + 0.013*"worker" + 0.009*"dollar"'), (3, '0.011*"united" + 0.011*"spread" + 0.010*"states" + 0.010*"guide" + 0.010*"oil" + 0.010*"open" + 0.010*"border" + 0.009*"place"'), (4, '0.018*"yeah" + 0.018*"china" + 0.016*"year" + 0.012*"life" + 0.011*"okay" + 0.011*"bad" + 0.011*"problem" + 0.010*"deal"'), (5, '0.031*"area" + 0.019*"jersey" + 0.013*"city" + 0.012*"system" + 0.012*"washington" + 0.011*"send" + 0.010*"louisiana" + 0.010*"african"'), (6, '0.017*"datum" + 0.012*"model" + 0.012*"community" + 0.012*"testing" + 0.011*"birx" + 0.010*"important" + 0.009*"fauci" + 0.009*

#### Visualize the topic model:

In [27]:
pyLDAvis.enable_notebook()
viz = pyLDAvis.gensim.prepare(lda, doc_term_matrix, dictionary, sort_topics=False)
viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.161141  0.116042       1        1  20.204775
1     -0.033082  0.061497       2        1  15.561588
2     -0.120134  0.237959       3        1  11.849381
3     -0.031240 -0.143245       4        1  13.886960
4     -0.244970 -0.093017       5        1  14.591358
5      0.186227 -0.006059       6        1   8.055758
6      0.082058 -0.173176       7        1  15.850186, topic_info=            Term        Freq       Total Category  logprob  loglift
2321  ventilator  554.000000  554.000000  Default  30.0000  30.0000
326     business  439.000000  439.000000  Default  29.0000  29.0000
0           area  445.000000  445.000000  Default  28.0000  28.0000
171   healthcare  404.000000  404.000000  Default  27.0000  27.0000
70         money  282.000000  282.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
143    different  107.219704  428.892761   Topic7  -5.0843   0.4557
83         start   98.287804  381.064087   Topic7  -5.1712   0.4869
421       impact   85.347961  167.701294   Topic7  -5.3124   1.1665
404          try   87.322350  285.533539   Topic7  -5.2895   0.6572
436         give   85.769516  310.604309   Topic7  -5.3075   0.5551

[459 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2457      6  0.977732      18
3775      3  0.970563     1st
3093      2  0.992219  abbott
233       1  0.115526    able
233       2  0.338125    able
...     ...       ...     ...
530       5  0.805298    yeah
530       7  0.061730    yeah
124       3  0.253465    year
124       5  0.637426    year
124       7  0.107911    year

[942 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7])

### Ideas for topic exploration:
 - Ventilators
 - China (spec: Wuhan)
 - PPE
 - Economy
 - Science